# Torontos Neighborhoods Clustering

Data of the neighborhoods was found in https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Preprocessing of data must be done

## Pre processing

First we must create a data frame with the data. 
We drop cells that don't contain value in the column Borough.
We merge duplicated postal codes that could be asigned to several Neighborhoods.

### Import libraries

In [1]:
import numpy as np
import pandas as pd
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from bs4 import BeautifulSoup
import lxml
from IPython.display import display_html

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\elfee\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.11.0               |   py38haa244fe_0        16.9 MB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        17.0 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0
  python_abi         conda-forge/win-64::python_abi-3.8-2_cp38

The following



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base -c defaults conda


C:\Users\elfee\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\elfee\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.2.0               |     pyhd8ed1ab_1         711 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1h             |       he774522_0         5.8 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.5 MB

The following NEW packages will be INSTALLED:

  altair             conda-forge

### Take a look at the html code sent as to know how to manipulate it

In [31]:
req = requests.get('http://zims-en.kiwix.campusafrica.gos.orange.com/wikipedia_en_all_nopic/A/List_of_postal_codes_of_Canada:_M')
sp = BeautifulSoup(req.text, 'html.parser')
print(sp)

<!DOCTYPE html>

<html class="client-js"><head><script src="/skin/block_external.js" type="text/javascript"></script>
<link href="/skin/jquery-ui/jquery-ui.min.css" rel="Stylesheet" type="text/css"/>
<link href="/skin/jquery-ui/jquery-ui.theme.min.css" rel="Stylesheet" type="text/css"/>
<link href="/skin/taskbar.css" rel="Stylesheet" type="text/css"/>
<script src="/skin/jquery-ui/external/jquery/jquery.js" type="text/javascript"></script>
<script src="/skin/jquery-ui/jquery-ui.min.js" type="text/javascript"></script>
<script>
  var jk = jQuery.noConflict();
  jk(function() {
  jk( "#kiwixsearchbox" ).autocomplete({

  source: "/suggest?content=wikipedia_en_all_nopic",
  dataType: "json",
  cache: false,

  select: function(event, ui) {
  jk( "#kiwixsearchbox" ).val(ui.item.value);
  jk( "#kiwixsearchform" ).submit();
  },

  });
  });

  /* cybook hack */
  if (navigator.userAgent.indexOf("bookeen/cybook") != -1) {
  	jk("html").addClass("cybook");
  }
</script>
<script async="" src="/

### Extract the data as text with Beautiful Soup

In [32]:
req = requests.get('http://zims-en.kiwix.campusafrica.gos.orange.com/wikipedia_en_all_nopic/A/List_of_postal_codes_of_Canada:_M').text
sp=BeautifulSoup(req,'lxml')
table = str(sp.table)
display_html(table,raw=True)

Postcode,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Not assigned
M8A,Not assigned,Not assigned
M9A,Downtown Toronto,Queen's Park


### Take away rows that have important cells not assigned

In [33]:
dfs = pd.read_html(table)
df=dfs[0]

dfpreClean = df[df.Borough != 'Not assigned']
dfClean = dfpreClean.groupby(['Postcode','Borough'], sort=False).agg(', '.join)
dfClean.reset_index(inplace=True)
dfClean['Neighbourhood'] = np.where(dfClean['Neighbourhood'] == 'Not assigned',dfClean['Borough'], dfClean['Neighbourhood'])
dfClean

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### Extract coordenates and add it to the corresponding postal code

In [34]:
coords = pd.read_csv('https://cocl.us/Geospatial_data')
coords.rename(columns={'Postal Code':'Postcode'},inplace=True)
dfwthCoords = pd.merge(dfClean,coords,on='Postcode')
dfwthCoords.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Restrict data to Toronto

In [35]:
dfToronto = dfwthCoords[dfwthCoords['Borough'].str.contains('Toronto',regex=False)]
dfToronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259


### Display in map

In [36]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(dfToronto['Latitude'],dfToronto['Longitude'],dfToronto['Borough'],dfToronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

## Start with clustering process

In [37]:
k=5
toronto_clustering = dfToronto.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
dfToronto.insert(0, 'Cluster Labels', kmeans.labels_)
dfToronto

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,0,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
9,1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,1,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,3,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,1,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,2,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,1,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
31,2,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259


### Graphic display of the cluster 

In [38]:
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, neighbourhood, cluster in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Neighbourhood'], dfToronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters